<a href="https://colab.research.google.com/github/yahiroyudai/DeepLearning2022Late/blob/main/RNN%2BExample_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##ライブラリのimport

In [1]:
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, LSTM, Dense

In [2]:
#from keras.layers.wrapers import TimeDistributed
from tensorflow.keras.layers import TimeDistributed

##データセット(imdb)の読み込み


In [3]:
from keras.datasets import imdb

In [4]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

##Numpyなどのツールの読み込み

In [5]:
import numpy as np
from keras.utils import to_categorical, np_utils

##データ数、バッチ数など、各種定数を定義する

In [6]:
train_reviews = 5000
valid_reviews = 100
max_features = 5000
Embedding_size = 256
srep_size = 5
batch_size = 32
index_from = 2
rnn_units = 128
epochs = 2
word_index_prev = {'<PAD>':0, '{START}':1, '<UNK>':2}

##IMDBデータセットを読み込む

In [7]:
(x_train,y_train), (x_test,y_test) = imdb.load_data( num_words=max_features, index_from=index_from )

##読み込んだデータを加工する(単語に関する情報を抜き出す)

In [8]:
word_index = { word:(index+index_from) 
for word, index in imdb.get_word_index().items() if (index + index_from) < max_features}
word_index.update(word_index_prev)

##加工した単語情報から辞書(ディクショナリ)を作成する

In [9]:
index_word = { index: word for word, index in word_index.items() }
print(index_word)

{1410: 'woods', 2347: 'hanging', 2291: 'woody', 2340: 'bringing', 1638: 'wooden', 4012: 'errors', 3232: 'dialogs', 361: 'kids', 180: 'want', 2105: 'travel', 3932: 'dinosaurs', 354: 'wrong', 4762: 'subplots', 1182: 'fit', 1929: 'screaming', 4289: 'fix', 301: 'effects', 4013: 'disturbed', 1287: 'purpose', 886: 'needed', 1299: 'master', 546: 'feeling', 1359: 'cinematic', 4990: 'tech', 659: 'saying', 3901: 'altogether', 1779: 'nicely', 4590: 'mummy', 774: 'lots', 875: 'nature', 3902: 'superficial', 2825: 'extent', 75: 'much', 4991: 'doubts', 3086: 'spin', 2578: 'hong', 4451: 'corporate', 4313: 'hal', 4894: 'ham', 68: 'had', 46: 'has', 2403: 'hat', 2292: 'crowd', 1323: 'bottom', 1183: 'starring', 3233: 'shoots', 3539: 'raped', 3977: 'explosions', 3628: 'chain', 2499: 'whoever', 3038: 'chair', 4519: 'ballet', 3398: 'jerk', 2896: 'locked', 2590: 'exact', 785: 'minute', 3385: 'unintentionally', 3842: 'cusack', 3629: 'opposed', 1044: 'following', 4520: 'convincingly', 4254: 'surfing', 1239: 'ji

##文章を表示する関数を定義する

In [10]:
def print_sentence(sentence):
  for index in sentence:
    print(index_word[index], end = " ")
  #ループ終わり
  print()
#関数定義終わり

##定義した関数print_sentenceで学習用データの1行目を表示してみる

In [11]:
print_sentence( x_train[0] )

{START} this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly <UNK> was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they

##学習用のtrainデータを学習用とバリデーション検証用に分ける

In [12]:
# train_reviews = 5000
# valid_reviews = 100 にわける
# 学習用 5000件
data_train = [ t for s in x_train[:train_reviews] for t in s ]
# バリデーション用 100
data_valid = [ t for s in x_train[train_reviews:train_reviews+valid_reviews] for t in s]

##バッチ処理のための関数を定義する

In [13]:
def batch_generator( data, batch_size, step_size ):
  seg_len = len(data) // batch_size
  steps_per_epoch = seg_len // step_size
  data_seg_list = np.asarray( [data[int(i*seg_len):int((i+1)*seg_len)]
                             for i in range(batch_size)] )
  i = 0
  while True:
    x = data_seg_list[:, int(i*step_size):int((i+1)*step_size)]
    y = np.asarray( [to_categorical(
    data_seg_list[j, int(i*step_size+1):int((i+1)*step_size+1)],max_features ) 
    for j in range(batch_size)])
    yield x, y 
    i +=1
    if i >= steps_per_epoch:
      i=0
    # if文おわり
  #whileループ終わり
# 関数def終わり


##学習モデルを組み立てる

In [14]:
w = Input(shape=(step_size, ), name='Input' ) # 入力層
x = Embedding( input_dim=max_features, output_dim=embedding_size, name='Enbedding')(w)
y = LSTM( units=rnn_units, return_sequences=True, dropout=0.5, recurrent_dropout=0.5, name='LSTM')(x)
w_next = TimeDistributed( Dense( units=max_features, activation='softmax', name='Dense'),name='TimeDistributed' )(y)
# モデルのインスタンスを生成
model = Model( inputs=[w], outputs=[w_next] )

NameError: ignored

##生成したモデルオブジェクトの形をSVGで表示

In [ ]:
SVG( model_to_dot( model, show_shapes=True,dpi=None ).create(prog='dot', format='svg') )

##モデルのコンパイル

In [ ]:
model.compile( optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'] )

##生成したモデルに学習させる（バッチごとに）


In [ ]:
# バッチ処理用にデータセットを加工
gen_train = batch_generator( data_train, batch_size, step_size )
gen_valid = batch_generator( data_valid, batch_size, step_size )

In [ ]:
# 各データのバッチごとのステップ数を学習時に指定するために算出
steps_per_epoch_train = len(data_train)/batch_size/step_size
steps_per_epoch_valid = len(data_valid)/batch_size/step_size

In [ ]:
# バッチ処理ごとにデータを学習とバリデーションする
model.fit_generator( generator=gen_train, steps_per_epoch=steps_per_epoch_train,epochs=epochs,
                    validation_data=gen_valid, validation_steps=steps_per_epoch_valid)

##学習したモデルを使ってランダムに文章を作らせる

In [ ]:
# 次にくる単語を選ぶ関数
def sample( preds, temperature=1.0 ):
  preds = np.log(preds) / temperature
  preds = np.exp(preds) / np.sum(np.exp(preds))
  choices = range(len(preds))
  return np.random.choice( choices, p=preds )
# 関数終わり

In [ ]:
# ランダムに文章を生成する関数
def sample_sentences( num_sentences, sample_sent_len=20 ):
  for x_test_i in x_test[:num_sentences]:
    x = np.zeros((1, step_size))
    sentence = x_test_i[:step_size]

    for i in range(sample_sent_len):
      for j, index in enumerate(sentence[-step_size:]):
        x[0, j] = index
      # for終わり
      preds = model.predict(x)[0][-1]
      next_index = sample(preds)
      sentence.append(next_index)
    # ループ終わり
    print_sentence(sentence)
  # ループ終わり
# 関数おわり